<a href="https://colab.research.google.com/github/lkhok22/ML-hw4/blob/main/face-recognition-EmotionCNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
! pip install kaggle

In [3]:
import wandb
wandb.login()
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
!kaggle competitions download -c challenges-in-representation-learning-facial-expression-recognition-challenge
!unzip -q challenges-in-representation-learning-facial-expression-recognition-challenge.zip

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: lkhok22 (lkhok22-free-university-of-tbilisi-) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


 82% 235M/285M [00:00<00:00, 674MB/s] 
100% 285M/285M [00:05<00:00, 58.7MB/s]


In [4]:
import pandas as pd
import numpy as np

In [5]:
train_df_t = pd.read_csv('train.csv')
test_df_t = pd.read_csv('test.csv')

In [6]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
import torchvision.transforms as transforms
from sklearn.metrics import accuracy_score, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
import wandb
import os
from PIL import Image

# Set random seed for reproducibility
torch.manual_seed(42)
np.random.seed(42)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [7]:
# Custom Dataset for FER2013
class FER2013Dataset(Dataset):
    def __init__(self, dataframe, transform=None):
        self.dataframe = dataframe
        self.transform = transform
        self.classes = ['Angry', 'Disgust', 'Fear', 'Happy', 'Sad', 'Surprise', 'Neutral']

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        # Pixels are stored as a string, convert to numpy array
        pixels = np.fromstring(self.dataframe.iloc[idx]['pixels'], dtype=int, sep=' ')
        image = pixels.reshape(48, 48).astype('float32')
        image = Image.fromarray(image)  # Convert to PIL Image for transforms
        label = int(self.dataframe.iloc[idx]['emotion'])

        if self.transform:
            image = self.transform(image)

        return image, label

# Data Preprocessing
train_transform = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5], std=[0.5])
])

val_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5], std=[0.5])
])


In [8]:

# Load and split data
train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')

# Create small dataset (20 images) for overfitting test
small_df = train_df.sample(n=20, random_state=42)

# Split train_df into train and validation
val_df = train_df.sample(frac=0.2, random_state=42)
train_df = train_df.drop(val_df.index)

train_dataset = FER2013Dataset(train_df, transform=train_transform)
val_dataset = FER2013Dataset(val_df, transform=val_transform)
small_dataset = FER2013Dataset(small_df, transform=train_transform)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=64, shuffle=False)
small_loader = DataLoader(small_dataset, batch_size=10, shuffle=True)


In [9]:
# Define Advanced CNN
class EmotionCNN(nn.Module):
    def __init__(self):
        super(EmotionCNN, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(1, 64, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.BatchNorm2d(64),
            nn.Conv2d(64, 64, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Dropout(0.25),

            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.BatchNorm2d(128),
            nn.Conv2d(128, 128, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Dropout(0.25),

            nn.Conv2d(128, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.BatchNorm2d(256),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Dropout(0.5),
        )
        self.classifier = nn.Sequential(
            nn.Linear(256 * 6 * 6, 512),
            nn.ReLU(inplace=True),
            nn.Dropout(0.5),
            nn.Linear(512, 7)
        )

    def forward(self, x):
        x = self.features(x)
        x = x.view(x.size(0), -1)
        x = self.classifier(x)
        return x


In [10]:

# Training and Evaluation Functions
def train_model(model, train_loader, val_loader, criterion, optimizer, num_epochs, run_name, overfit_test=False):
    wandb.init(project="fer2013", name=run_name, config={
        "architecture": "EmotionCNN",
        "epochs": num_epochs,
        "learning_rate": optimizer.param_groups[0]['lr'],
        "batch_size": train_loader.batch_size,
        "dataset_size": len(train_loader.dataset),
        "overfit_test": overfit_test
    })

    best_val_acc = 0.0
    for epoch in range(num_epochs):
        model.train()
        train_loss, train_correct, train_total = 0.0, 0, 0
        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            train_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            train_total += labels.size(0)
            train_correct += (predicted == labels).sum().item()

        train_loss /= len(train_loader)
        train_acc = train_correct / train_total

        # Validation
        model.eval()
        val_loss, val_correct, val_total = 0.0, 0, 0
        all_preds, all_labels = [], []
        with torch.no_grad():
            for images, labels in val_loader:
                images, labels = images.to(device), labels.to(device)
                outputs = model(images)
                loss = criterion(outputs, labels)
                val_loss += loss.item()
                _, predicted = torch.max(outputs, 1)
                val_total += labels.size(0)
                val_correct += (predicted == labels).sum().item()
                all_preds.extend(predicted.cpu().numpy())
                all_labels.extend(labels.cpu().numpy())

        val_loss /= len(val_loader)
        val_acc = val_correct / val_total

        # Log metrics to Wandb
        wandb.log({
            "epoch": epoch + 1,
            "train_loss": train_loss,
            "train_accuracy": train_acc,
            "val_loss": val_loss,
            "val_accuracy": val_acc
        })

        # Plot and log confusion matrix at the final epoch
        if epoch == num_epochs - 1:
            cm = confusion_matrix(all_labels, all_preds)
            plt.figure(figsize=(10, 8))
            sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=['Angry', 'Disgust', 'Fear', 'Happy', 'Sad', 'Surprise', 'Neutral'],
                        yticklabels=['Angry', 'Disgust', 'Fear', 'Happy', 'Sad', 'Surprise', 'Neutral'])
            plt.title("Confusion Matrix")
            plt.ylabel("True Label")
            plt.xlabel("Predicted Label")
            wandb.log({"confusion_matrix": wandb.Image(plt)})
            plt.close()

        # Save best model
        if val_acc > best_val_acc:
            best_val_acc = val_acc
            torch.save(model.state_dict(), f"best_model_{run_name}.pth")
            wandb.save(f"best_model_{run_name}.pth")

    wandb.finish()
    return best_val_acc


In [11]:

# Overfitting Test on Small Dataset
model = EmotionCNN().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
print("Training on small dataset (20 images) to test overfitting...")
best_val_acc_small = train_model(model, small_loader, val_loader, criterion, optimizer, num_epochs=10, run_name="small_dataset_test", overfit_test=True)


Training on small dataset (20 images) to test overfitting...


epoch,▁▂▃▃▄▅▆▆▇█
train_accuracy,▁▂▄▄▆▃█▅▇█
train_loss,█▇▄▅▄▃▂▁▂▁
val_accuracy,▆▇▆▆▁▃▅▅▇█
val_loss,▁▁▂▅▆▅▅▆▇█
epoch,10
train_accuracy,0.7
train_loss,1.37614
val_accuracy,0.17868
val_loss,10.79335


In [12]:

# Full Training
model = EmotionCNN().to(device)  # Reset model
optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-4)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.1)
print("Training on full dataset...")
best_val_acc_full = train_model(model, train_loader, val_loader, criterion, optimizer, num_epochs=30, run_name="full_dataset_cnn")


Training on full dataset...


epoch,▁▁▁▂▂▂▂▃▃▃▃▄▄▄▄▅▅▅▅▆▆▆▆▇▇▇▇███
train_accuracy,▁▃▄▄▅▅▅▆▆▆▆▆▆▆▇▇▇▇▇▇▇▇▇███████
train_loss,█▆▅▅▄▄▄▃▃▃▃▃▃▃▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁
val_accuracy,▁▃▄▅▆▆▆▆▆▇▇▇▇▇▇▇▇▇▇▇██████████
val_loss,█▅▄▃▃▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁
epoch,30
train_accuracy,0.62372
train_loss,1.00033
val_accuracy,0.62557
val_loss,0.99818
